# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
agg_df = data.groupby(by=['CustomerID', 'ProductName'], as_index=0)['Quantity'].sum()
agg_df.head(5)

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
pt_df = agg_df.pivot_table(values='Quantity', index='ProductName', columns='CustomerID', aggfunc='sum').fillna(0)
cust_pt_df = pt_df.T

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
dist_calculation = pdist(X=cust_pt_df, metric='euclidean')
len(dist_calculation)

499500

In [7]:
dist_distribution = squareform(dist_calculation)
len(dist_distribution)

1000

In [8]:
products = data['ProductName'].sort_values().unique()
customers = data['CustomerID'].sort_values().unique()

In [9]:
euclid_dist = pd.DataFrame(dist_distribution,
                           index=customers, 
                           columns=customers)

euclid_dist.head(5)

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936


In [10]:
euclid_dist_norm = pd.DataFrame(1/(1 + squareform(pdist(X=cust_pt_df, metric='euclidean'))),
                                index=customers,
                                columns=customers)

euclid_dist_norm.head(5)

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [70]:
euclid_dist_norm[33].sort_values(ascending=False).iloc[:5]

33      1.000000
3317    0.087047
3535    0.087047
264     0.087047
2503    0.085983
Name: 33, dtype: float64

In [91]:
similar_c = euclid_dist_norm[33].sort_values(ascending=False).iloc[1:6].index
list(similar_c)

[3317, 3535, 264, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [92]:
similar_buy = agg_df[agg_df['CustomerID'].isin(similar_c)]

In [93]:
similar_buy.head(5)

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [94]:
ranked_similar_buy = similar_buy.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).reset_index()
ranked_similar_buy.head()

,ProductName,Quantity
0,Butter - Unsalted,3
1,Wine - Ej Gallo Sierra Valley,3
2,Towels - Paper / Kraft,3
3,Soup - Campbells Bean Medley,3
4,Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [95]:
merged_ranked_similar_buy = ranked_similar_buy.merge(pt_df.reset_index()[['ProductName', 33]])

In [96]:
merged_ranked_similar_buy[merged_ranked_similar_buy[33] == 0].head(5).reset_index()

,index,ProductName,Quantity,33
0,0,Butter - Unsalted,3,0.0
1,1,Wine - Ej Gallo Sierra Valley,3,0.0
2,3,Soup - Campbells Bean Medley,3,0.0
3,4,Wine - Blue Nun Qualitatswein,3,0.0
4,6,Chicken - Soup Base,2,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [97]:
recommendations_dict = {'CustomerID': [],'Product_01': [], 'Product_02': [],
                       'Product_03': [], 'Product_04': [], 'Product_05': []}

for i in customers:
    alike = euclid_dist_norm[i].sort_values(ascending=False).iloc[1:6].index
    alike_df = agg_df[agg_df['CustomerID'].isin(alike)]
    ranked = alike_df.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).reset_index()
    merged_rank = ranked.merge(pt_df.reset_index()[['ProductName', i]])
    merged_rank = merged_rank[merged_rank[i] == 0].head(5).reset_index()
    
    recommendations_dict['CustomerID'].append(i)
    recommendations_dict['Product_01'].append(merged_rank['ProductName'][0])
    recommendations_dict['Product_02'].append(merged_rank['ProductName'][1])
    recommendations_dict['Product_03'].append(merged_rank['ProductName'][2])
    recommendations_dict['Product_04'].append(merged_rank['ProductName'][3])
    recommendations_dict['Product_05'].append(merged_rank['ProductName'][4])

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [98]:
rec_df = pd.DataFrame(recommendations_dict)
rec_df.head(5)

,CustomerID,Product_01,Product_02,Product_03,Product_04,Product_05
0,33,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein,Chicken - Soup Base
1,200,Soup - Campbells Bean Medley,Muffin - Carrot Individual Wrap,Bay Leaf,Pork - Kidney,Wanton Wrap
2,264,Soupfoamcont12oz 112con,Wine - Two Oceans Cabernet,Bread - Italian Roll With Herbs,"Veal - Inside, Choice","Fish - Scallops, Cold Smoked"
3,356,Butter - Unsalted,"Veal - Inside, Choice","Beets - Candy Cane, Organic","Nut - Chestnuts, Whole",Lamb - Ground
4,412,Olive - Spread Tapenade,Sprouts - Baby Pea Tendrils,Wine - Blue Nun Qualitatswein,"Pepper - Black, Whole",Soup - Campbells Bean Medley


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [99]:
cosine_dist_norm = pd.DataFrame(1/(1 + squareform(pdist(X=cust_pt_df, metric='cosine'))),
                                index=customers,
                                columns=customers)

In [100]:
recommendations_dict_2 = {'CustomerID': [],'Product_01': [], 'Product_02': [],
                       'Product_03': [], 'Product_04': [], 'Product_05': []}

for i in customers:
    alike = cosine_dist_norm[i].sort_values(ascending=False).iloc[1:6].index
    alike_df = agg_df[agg_df['CustomerID'].isin(alike)]
    ranked = alike_df.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).reset_index()
    merged_rank = ranked.merge(pt_df.reset_index()[['ProductName', i]])
    merged_rank = merged_rank[merged_rank[i] == 0].head(5).reset_index()
    
    recommendations_dict_2['CustomerID'].append(i)
    recommendations_dict_2['Product_01'].append(merged_rank['ProductName'][0])
    recommendations_dict_2['Product_02'].append(merged_rank['ProductName'][1])
    recommendations_dict_2['Product_03'].append(merged_rank['ProductName'][2])
    recommendations_dict_2['Product_04'].append(merged_rank['ProductName'][3])
    recommendations_dict_2['Product_05'].append(merged_rank['ProductName'][4])

In [101]:
rec_df_2 = pd.DataFrame(recommendations_dict_2)
rec_df_2.head(5)

,CustomerID,Product_01,Product_02,Product_03,Product_04,Product_05
0,33,Knife Plastic - White,"Soup - Campbells, Beef Barley",Onions - Cippolini,Tea - Herbal Sweet Dreams,Muffin - Zero Transfat
1,200,Longos - Grilled Salmon With Bbq,Snapple Lemon Tea,General Purpose Trigger,"Thyme - Lemon, Fresh",Tomatoes Tear Drop
2,264,Pickerel - Fillets,"Water - Mineral, Natural",Snapple - Iced Tea Peach,Wine - Ej Gallo Sierra Valley,French Pastry - Mini Chocolate
3,356,Bread - English Muffin,Olive - Spread Tapenade,Bagel - Plain,Pork - Hock And Feet Attached,Wiberg Super Cure
4,412,"Salsify, Organic",Durian Fruit,Wine - Hardys Bankside Shiraz,Bread - Raisin Walnut Oval,Gatorade - Xfactor Berry
